# Tree LDPC Code
This Notebook serves to generate a LDPC Code without cycles. <br>
One specifies n, m, $d_{c,max}$ and $d_{v,max}$.

In [15]:
import numpy as np
from collections import deque

In [16]:
dc_max = 5
dv_max = 5
n = 16
m = 8

### Tree
V = {vertices}, E = {edges}, S = {sockets} <br>
|E| = |V| - 1 <br>
|S| = 2 |E| <br>
<br>
### Bipartite Graph
|E| sockets for variable nodes + |E| sockets for check nodes

In [17]:
def randomNodeDegrees(node_cnt, socket_cnt, d_max):
    '''
    radomly distributes sockets on nodes
    returns an sorted array with random degrees for @node_cnt nodes such that the degrees sumup to @socket_cnt
    while ensuring that: 1 <= degree <= d_max
    '''
    degrees = np.empty(node_cnt, dtype=int)
    available_sockets = socket_cnt
    for node_idx in range(node_cnt):
        min_degree = max(1, available_sockets - (node_cnt - node_idx - 1) * d_max)
        max_degree = min(d_max, available_sockets - (node_cnt - node_idx - 1))
        degrees[node_idx] = np.random.randint(min_degree, max_degree + 1, dtype=int)
        available_sockets -= degrees[node_idx]
    return np.sort(degrees)[::-1]

In [18]:
E = n + m - 1
varnode_sockets = randomNodeDegrees(n, E, dv_max)
checknode_sockets = randomNodeDegrees(m, E, dc_max)

# bfs to populate H, ascending sorted arrays ensure minimal depth
H = np.zeros((m, n), dtype=np.int8)
q = deque()
q.append(0) # add root node
varnode_cnt = 1
checknode_cnt = 0
while len(q) > 0:
    varnode_idx = q[0]
    q.popleft()
    for checknode in range(varnode_sockets[varnode_idx] - (0 if varnode_idx == 0 else 1)): # account for parent in case of non-root nodes
        checknode_idx = checknode_cnt + checknode
        H[checknode_idx,varnode_idx] = 1
        q.extend(range(varnode_cnt, varnode_cnt + checknode_sockets[checknode_idx] - 1))
        H[checknode_idx,varnode_cnt:varnode_cnt + checknode_sockets[checknode_idx] - 1] = 1
        varnode_cnt += checknode_sockets[checknode_idx] - 1
    checknode_cnt += varnode_sockets[varnode_idx] - (0 if varnode_idx == 0 else 1) # account for parent in case of non-root nodes


In [19]:
print(H)
np.save('codes/random_acyclic_LDPC', H)

[[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [20]:
H_load = np.load('codes/random_acyclic_LDPC.npy')
np.all(H == H_load)

True

In [21]:
#import galois -> Galois package is evil it damages the max function ...
GF = galois.GF(2)
G = GF(H).null_space()
H_min = G.null_space()
print(H_min)

NameError: name 'galois' is not defined

In [ ]:
def max(H):
    # implementiere BFS oder DFS aus Informatik Olympiade
    pass